Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Activation,MaxPooling2D,Flatten,BatchNormalization,Dropout,InputLayer
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Function for creating CNN Model using parameters given

In [11]:
def create_cnn_model(filters, fil_size, dense_neuron, batch_norm, dropout, act, padd, lr):
    keras.backend.clear_session()
    model = Sequential()
    model.add(InputLayer(input_shape=(227,227,3))) #Adding Input layer with image size as(227,227) with RGB channel

    #Adding 5 convolution Layers in a loop
    for i in range(5):
        model.add(Conv2D(filters[i],fil_size[i],padding=padd))
        if batch_norm:
            model.add(BatchNormalization())   
        model.add(Activation(act))
        model.add(MaxPooling2D(pool_size=(2,2)))
    
    #Adding the Dense layers as per the parameters passed
    model.add(Flatten())
    model.add(Dense(dense_neuron))
    model.add(Dropout(dropout))
    model.add(Activation("relu"))

    model.add(Dense(10,activation='softmax'))
    adam = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

    return model

Definition Function for preparing DATA:-

In [13]:
def dataProcess(BATCH_SIZE, data_aug=True):
    if(data_aug):
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=90,
            shear_range=0.2,
            zoom_range=0.2,
            validation_split=0.1,
            horizontal_flip=True)
    else:
        train_datagen = ImageDataGenerator( rescale=1./255, validation_split=0.1)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/DL_DATA_iNat/train',
        subset='training',
        target_size=(227,227),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
        seed = 45)

    val_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/DL_DATA_iNat/train',
        subset = 'validation',
        target_size=(227,227),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
        seed = 45)

    test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/DL_DATA_iNat/val',
        target_size=(227,227),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
    return train_generator, val_generator, test_generator
    

In [14]:
fil=[32,64,64,128,128]
fils=[(7,7),(7,7),(7,7),(7,7),(7,7)]
d=0.15
batch_norm=True
epochs=5,
d_n=64
ler=0.001
trial = create_cnn_model(fil,fils,d_n,batch_norm,d,'relu',"same",ler)

In [ ]:
trial.fit(
          train_gen,
          steps_per_epoch = train_gen=.samples // 256,
          validation_data = val_gen, 
          validation_steps = val_gen.samples // 256,
          epochs = 5
        )

In [ ]:
trial.evaluate(test_generator,batch_size = 32)

WandB install and login ---------------------- ---------- -------

In [ ]:
!pip install wandb


In [8]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [15]:
#--------Fetching dataset---------------------------------

BATCH_SIZE = 32
train_gen, val_gen, test_gen = dataProcess(BATCH_SIZE, True)

Found 9018 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2009 images belonging to 10 classes.


In [16]:
sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': { 'values': [12]
        },
        'filters': {
            'values': [[16,32,64,128,256],
                       [32,64,128,256,256],
                       [64,32,16,8,4],
                       [32,16,8,4,2],
                       [64,64,64,64,64],
                       [32,32,32,32,32],
                       [48,48,48,48,48]
                      ]
        },
        'filter_size': {
            'values': [[(7,7),(7,7),(7,7),(7,7),(7,7)],
                       [(5,5),(5,5),(5,5),(5,5),(5,5)],
                       [(3,3),(3,3),(3,3),(3,3),(3,3)],
                       [(2,2),(2,2),(2,2),(2,2),(2,2)]
                      ]
        },
        'dense_neuron': {
            'values': [128,256,512]
        },
        'batch_norm': {
            'values': [True,False]
        },
        'dropout': {
            'values': [0.15,0.2]
        },
        'lr': {
            'values': [1e-4,1e-5]
        }
    }
}

In [17]:
import wandb
from wandb.keras import WandbCallback

def train():
    config_defaults = {
        "filters": [32,64,64,128,128],
        "filter_size": [(7,7),(7,7),(7,7),(7,7),(7,7)],
        "dropout": 0.15,
        "batch_norm": True,
        "epochs": 5,
        "dense_neuron": 64,
        "lr": 0.001
    }

    wandb.init(config=config_defaults,name="CS6910-A2-P_A")
    config = wandb.config
    wandb.run.name = "flt_{}_fltsz_{}_do_{}_bn_{}_dsz_{}_lr_{}".format(config.filters,\
                                                      config.filter_size[0],\
                                                      config.dropout,\
                                                      config.batch_norm,\
                                                      config.dense_neuron,\
                                                      config.lr)
    wandb.run.save()

    model = create_cnn_model(config.filters,config.filter_size,config.dense_neuron,config.batch_norm,config.dropout,'relu',"same",config.lr)

    trained_arti = model.fit(
                              train_gen,
                              steps_per_epoch = train_gen.samples // BATCH_SIZE,
                              validation_data = val_gen, 
                              validation_steps = val_gen.samples // BATCH_SIZE,
                              epochs = config.epochs, 
                              callbacks=[WandbCallback(monitor='val_accuracy',mode='max')]
                            )
    model.evaluate(
                    test_gen,
                    batch_size = BATCH_SIZE,
                    callbacks=[WandbCallback()]
                  )
    #model.save('/content/drive/MyDrive/DL_DATA_iNat/best_model.h5)
    wandb.finish()
    return model, trained_arti

In [19]:
sweep_id = wandb.sweep(sweep_config,entity="pandp",project = 'cs6910-As-2')

Create sweep with ID: 9khqsoyk
Sweep URL: https://wandb.ai/pandp/cs6910-As-2/sweeps/9khqsoyk


Create sweep with ID: 9khqsoyk
Sweep URL: https://wandb.ai/pandp/cs6910-As-2/sweeps/9khqsoyk

In [ ]:
wandb.agent(sweep_id, train, count=40)

wandb: Agent Starting Run: m1e46dy7 with config:
wandb: 	batch_norm: False
wandb: 	dense_neuron: 512
wandb: 	dropout: 0.15
wandb: 	epochs: 12
wandb: 	filter_size: [[5, 5], [5, 5], [5, 5], [5, 5], [5, 5]]
wandb: 	filters: [64, 64, 64, 64, 64]
wandb: 	lr: 1e-05
wandb: Currently logged in as: pranab045 (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Epoch 1/12
 68/281 [======>.......................] - ETA: 29:13 - loss: 2.3042 - accuracy: 0.1051